<a href="https://colab.research.google.com/github/preethamgoud9/-Nanogpt/blob/main/ann_fashion_mnist_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
import matplotlib.pyplot as plt

In [62]:
torch.manual_seed(42)

In [63]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"using devide:{device}")

using devide:cuda


In [64]:
df = pd.read_csv("fashion-mnist_train.csv")
df.shape

(60000, 785)

In [65]:
#train test split
x = df.iloc[:,1:].values
y = df.iloc[:,0].values

In [66]:
x_train,x_test,y_train,y_test = train_test_split(x,y , test_size=0.2 , random_state=42)

In [67]:
#scaling the features, basically we are normalizing the values to an smaller intervals (0-1)
x_train = x_train/255.0
x_test = x_test/255.0

In [68]:
#creating custom dataset
class CustomDataset(Dataset):

    def __init__(self,features,labels):

        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels,dtype=torch.long)

    def __len__(self):
        return len(self.features)

    def __getitem__(self,index):
        return self.features[index],self.labels[index]

In [69]:
#create train_dataset object
train_dataset = CustomDataset(x_train,y_train)

In [70]:
len(train_dataset)

48000

In [71]:
#create train_dataset object
test_dataset = CustomDataset(x_test,y_test)

In [72]:
len(test_dataset)

12000

In [73]:
#creating train and test loader
train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=32,shuffle=False)

In [74]:
#define nn class
class MyNN(nn.Module):

    def __init__(self,num_features):
        super().__init__()

        self.model = nn.Sequential(
            nn.Linear(num_features,128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(128,64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(64,10)
        )
    def forward(self,x):
        return self.model(x)

In [75]:
#set learning rate and epochs
learning_rate = 0.1
epochs = 100

In [76]:
#instantiate the model
model = MyNN(x_train.shape[1])
model = model.to(device)

#loss function
criterion = nn.CrossEntropyLoss()

#optimizer
optimizer = optim.SGD(model.parameters(),lr =learning_rate, weight_decay=1e-4)

In [77]:
#training loop
for epoch in range(epochs):

    total_epoch_loss = 0

    for batch_features, batch_labels in train_loader:

        #move data to gpu
        batch_features, batch_labels =batch_features.to(device), batch_labels.to(device)

        #forward pass
        outputs = model(batch_features)
        #loss calculation
        loss =criterion(outputs,batch_labels)
        #backpropagation
        optimizer.zero_grad()
        loss.backward()
        #update the parameters
        optimizer.step()

        total_epoch_loss = total_epoch_loss + loss.item()

    avg_loss = total_epoch_loss/len(train_loader)
    print(f"Epoch:{epoch +1} and Loss : {avg_loss}")


Epoch:1 and Loss : 0.6249084644317627
Epoch:2 and Loss : 0.4919969036479791
Epoch:3 and Loss : 0.455620892132322
Epoch:4 and Loss : 0.4338064221441746
Epoch:5 and Loss : 0.41715061252812546
Epoch:6 and Loss : 0.4056409275531769
Epoch:7 and Loss : 0.39416088837385177
Epoch:8 and Loss : 0.3858017429014047
Epoch:9 and Loss : 0.37439838365713757
Epoch:10 and Loss : 0.37257265872259937
Epoch:11 and Loss : 0.36783314737677575
Epoch:12 and Loss : 0.35720522227386636
Epoch:13 and Loss : 0.3505206037561099
Epoch:14 and Loss : 0.3449219425916672
Epoch:15 and Loss : 0.34472561853379013
Epoch:16 and Loss : 0.337321236188213
Epoch:17 and Loss : 0.3344038932522138
Epoch:18 and Loss : 0.3302020431111256
Epoch:19 and Loss : 0.33063985271751883
Epoch:20 and Loss : 0.3262277199824651
Epoch:21 and Loss : 0.32084967653950053
Epoch:22 and Loss : 0.3183093220243851
Epoch:23 and Loss : 0.3225850307941437
Epoch:24 and Loss : 0.31459669538835683
Epoch:25 and Loss : 0.3134305283402403
Epoch:26 and Loss : 0.3142

In [78]:
#set the model to eval mode
model.eval()

MyNN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.3, inplace=False)
    (8): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [79]:
#evaluation
total = 0
correct = 0
with torch.no_grad():
    for batch_features,batch_labels in test_loader:

        #move data to gpu
        batch_features, batch_labels =batch_features.to(device), batch_labels.to(device)

        outputs = model(batch_features)
        _,predicted = torch.max(outputs,1)

        total = total + batch_features.shape[0]
        correct = correct + (predicted == batch_labels).sum().item()

    accuracy = correct/total
    print(accuracy)



0.88325
